In [1]:
import networkx as nx;
import pandas as pd;
import gurobipy as gp;
from gurobipy import GRB;
import csv;
import sys;
import time;
from datetime import datetime;
import math;
import random;
import itertools;
import numpy as np;

In [2]:
def solver_optimistic (G, budget, s, t, A):
    
    start_time = time.time(); 
    networkCSV = network+'.csv';
    
    opt_model = gp.Model("Optimistic"); 
    
    gamma = opt_model.addVars(G.edges, vtype=GRB.BINARY, name = "gamma"); 
    z = opt_model.addVars(G.edges, vtype=GRB.BINARY, name = "z"); 
    x = opt_model.addVars(G.edges, vtype=GRB.CONTINUOUS, name = "x", lb = 0, ub = GRB.INFINITY);
    theta = opt_model.addVars(G.edges, vtype=GRB.CONTINUOUS, name = "theta", lb = 0, ub = 1); 
    alpha = opt_model.addVars(G.nodes, vtype=GRB.CONTINUOUS, name = "alpha", lb = 0, ub = 1); 
    beta = opt_model.addVars(G.edges, vtype=GRB.CONTINUOUS, name = "beta", lb = 0, ub = 1); 
    
    
    opt_model.addConstr(gp.quicksum(G.edges[i,j]['cost']*gamma[i,j] for i,j in G.edges) <= budget);
    
    opt_model.addConstr(gp.quicksum(G.edges[i,j]['capacity']*beta[i,j] for i,j in G.edges) <= x[t,s]);
    
    opt_model.addConstr(alpha[t] - alpha[s] >= 1);
    
    for i,j in G.edges:
        if G.edges[i,j]['special'] == 1:
            opt_model.addConstr(x[i,j] - G.edges[i,j]['capacity']*z[i,j] <= 0);
        
        opt_model.addConstr(alpha[i] - alpha[j] + theta[i,j] >= 0);
        opt_model.addConstr(beta[i,j] + gamma[i,j] - theta[i,j] >= 0);
        opt_model.addConstr(x[i,j] - G.edges[i,j]['capacity']*(1-gamma[i,j]) <= 0);
            
    opt_model.addConstrs(gp.quicksum(x[v,u] for u in G.successors(v)) -
                     gp.quicksum(x[u,v] for u in G.predecessors(v)) == 0 for v in G.nodes);
            
    opt_model.setObjective(gp.quicksum(G.edges[i,j]['special']*z[i,j] for i,j in G.edges), GRB.MINIMIZE);
    
    opt_model.setParam("IntegralityFocus",1);
    #opt_model.setParam('TimeLimit', T_Limit); 
    opt_model.update();
    
    opt_model.setParam("OutputFlag", 0);
    opt_model.optimize();
    
    
    status = opt_model.status;
    end_time = time.time();
    
    print('Optimistic LB: %g' %opt_model.objVal);
    run_time = round(end_time - start_time, 2);
    print('run time: %g sec \n' %run_time);
    
    # ~~~~~~~~~~~~~~~~~ OPTIMISTIC MIP START FILE~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    mst_file = open('MST_Optimistic_'+network+'_b'+str(budget)+'.mst', "w");
    
    mst_file.write('# gamma values \n');
    for i,j in G.edges:
        mst_file.write('%s %g \n' %(gamma[i,j].VarName, gamma[i,j].x));
    mst_file.write('#\n');
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
    return opt_model.objVal;

In [3]:
def MFNIP_Flow (network, budget, G, s, t):
    
    MFNIP_Flow = gp.Model("MFNIP_Flow"); 
    
    gamma = MFNIP_Flow.addVars(G.edges, vtype=GRB.BINARY); 
    x = MFNIP_Flow.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);
    theta = MFNIP_Flow.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = 1); 
    alpha = MFNIP_Flow.addVars(G.nodes, vtype=GRB.CONTINUOUS, lb = 0, ub = 1); 
    beta = MFNIP_Flow.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = 1); 
    
    
    MFNIP_Flow.addConstr(gp.quicksum(G.edges[i,j]['cost']*gamma[i,j] for i,j in G.edges) <= budget);
    
    MFNIP_Flow.addConstr(x[t,s] >= gp.quicksum(G.edges[i,j]['capacity']*beta[i,j] for i,j in G.edges));
    
    MFNIP_Flow.addConstr(alpha[t] - alpha[s] >= 1);
    
    for i,j in G.edges:
        MFNIP_Flow.addConstr(alpha[i] - alpha[j] + theta[i,j] >= 0);
        MFNIP_Flow.addConstr(beta[i,j] + gamma[i,j] - theta[i,j] >= 0);
        MFNIP_Flow.addConstr(x[i,j] - G.edges[i,j]['capacity']*(1-gamma[i,j]) <= 0);
            
    MFNIP_Flow.addConstrs(gp.quicksum(x[v,u] for u in G.successors(v)) -
                     gp.quicksum(x[u,v] for u in G.predecessors(v)) == 0 for v in G.nodes);
    
    
    MFNIP_Flow.setObjective(x[t,s], GRB.MINIMIZE);
    
    MFNIP_Flow.setParam("IntegralityFocus",1);
    MFNIP_Flow.update();
    MFNIP_Flow.setParam("OutputFlag", 0);
    MFNIP_Flow.optimize();
    
    return MFNIP_Flow.objVal;

In [4]:
def MaxFlow (G, s, t):
    
    maxFlow = gp.Model("MaxFlow"); 
    
    x = maxFlow.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);
    
    maxFlow.addConstrs(gp.quicksum(x[j,i] for i in G.successors(j))-
                       gp.quicksum(x[i,j] for i in G.predecessors(j)) == 0 for j in G.nodes);
    
    for i,j in G.edges:
        maxFlow.addConstr(x[i,j] <= G.edges[i,j]['capacity']);
        
    maxFlow.setObjective(x[t,s], GRB.MAXIMIZE);
    
    maxFlow.setParam("OutputFlag", 0);
    maxFlow.update();
    maxFlow.optimize();
                          
    return maxFlow.objVal;    

In [5]:
def solver_MFNIP (budget, network):
    
    start_time = time.time();
    networkCSV = 'Modified_'+network+'.csv';

    # Reading network file
    with open(networkCSV, newline='') as f:
        reader = csv.reader(f);
        row1 = next(reader);
        s = int(row1[0]);             # Source node
        t = int(row1[1]);             # Sink node
        calA_level = int(row1[2]);    # Level of special arcs
    
        G = nx.DiGraph();
        data = pd.read_csv(networkCSV, skiprows=1, header=None);
        n_edge = len(data.index+1);
    
        for i in range(n_edge): 
            G.add_edge(data.iat[i,0], data.iat[i,1], capacity= data.iat[i,2], 
                    cost=data.iat[i,3], special=data.iat[i,4], level=data.iat[i,5]);  
   
    MFNIP_model = gp.Model("MFNIP"); 
    
    gamma = MFNIP_model.addVars(G.edges, vtype=GRB.BINARY, name="gamma"); 
    x = MFNIP_model.addVars(G.edges, vtype=GRB.CONTINUOUS, name="x", lb = 0, ub = GRB.INFINITY);
    theta = MFNIP_model.addVars(G.edges, vtype=GRB.CONTINUOUS, name="theta", lb = 0, ub = 1); 
    alpha = MFNIP_model.addVars(G.nodes, vtype=GRB.CONTINUOUS, name="alpha", lb = 0, ub = 1); 
    beta = MFNIP_model.addVars(G.edges, vtype=GRB.CONTINUOUS, name="beta", lb = 0, ub = 1); 
    
    
    MFNIP_model.addConstr(gp.quicksum(G.edges[i,j]['cost']*gamma[i,j] for i,j in G.edges) <= budget);
    
    MFNIP_model.addConstr(x[t,s] >= gp.quicksum(G.edges[i,j]['capacity']*beta[i,j] for i,j in G.edges));
    
    MFNIP_model.addConstr(alpha[t] - alpha[s] >= 1);
    
    for i,j in G.edges:
        MFNIP_model.addConstr(alpha[i] - alpha[j] + theta[i,j] >= 0);
        MFNIP_model.addConstr(beta[i,j] + gamma[i,j] - theta[i,j] >= 0);
        MFNIP_model.addConstr(x[i,j] - G.edges[i,j]['capacity']*(1-gamma[i,j]) <= 0);
            
    MFNIP_model.addConstrs(gp.quicksum(x[v,u] for u in G.successors(v)) -
                     gp.quicksum(x[u,v] for u in G.predecessors(v)) == 0 for v in G.nodes);
     
    
    MFNIP_model.setObjective(x[t,s], GRB.MINIMIZE);
    
    MFNIP_model.setParam("IntegralityFocus",1);
    MFNIP_model.update();

    MFNIP_model.setParam("OutputFlag", 0);
    MFNIP_model.optimize();
    
    
    # ~~~~~~~~~~~~ MST FILE (ONLY GAMMA VALUES) ~~~~~~~~~~~~~~~~~~~~~~  ##
    mst_file = open('MST_MFNIP_'+network+'_b'+str(budget)+'.mst', "w");
    
    mst_file.write('# gamma values \n');
    for i,j in G.edges:
        mst_file.write('%s %g \n' %(gamma[i,j].VarName, gamma[i,j].x));
    mst_file.write('#\n');
    
    mst_file.close();
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ##
    
    end_time = time.time();
    print('MFNIP UB: %g' %MFNIP_model.objVal);
    run_time = round(end_time - start_time, 2);
    print('run time: %g sec \n' %run_time);
    
    #return MFNIP_model.objVal;

In [6]:
def Max_InnerProblem(G, s, t, M, Gamma, XBar):
    
    maxInner = gp.Model("Inner"); 
    
    x = maxInner.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);
    z = maxInner.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = 1); 
    
    maxInner.addConstrs(gp.quicksum(x[j,i] for i in G.successors(j))-
                        gp.quicksum(x[i,j] for i in G.predecessors(j)) == 0 for j in G.nodes);
    
    
    for i,j in G.edges:
        maxInner.addConstr(x[i,j] <= (1-Gamma[i,j])*G.edges[i,j]['capacity']);
        if G.edges[i,j]['special'] == 1:
            maxInner.addConstr(x[i,j] >= (1/M)*z[i,j]);
    
    maxInner.addConstr(x[t,s] >= XBar[t,s]);
    
    maxInner.setObjective(gp.quicksum(z[i,j]*G.edges[i,j]['special'] for i,j in G.edges), GRB.MAXIMIZE);
    
    maxInner.setParam("OutputFlag", 0);
    maxInner.update();
    maxInner.optimize();
    
    status = maxInner.status;
    
    if status == 2:
        print('Inner Problem Status: Optimal');
        print('ObjVal = %g' % maxInner.objVal);
    elif status == 3:
        print('Inner Problem status: Infeasible');
    else:
        print('Not optimal! Status = %g' %status);
        
    print('\n');
    

In [7]:
def Dual_InnerProblem(G, s, t, M, Gamma, XBar):
    
    dual = gp.Model("Dual_InnerProblem"); 
    
    x = dual.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);
    z = dual.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = 1); 
    
    alpha = dual.addVars(G.nodes, vtype=GRB.CONTINUOUS, lb = -GRB.INFINITY, ub = GRB.INFINITY); 
    theta = dual.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY); 
    pi = dual.addVars(G.edges, vtype=GRB.CONTINUOUS, lb =-GRB.INFINITY, ub=0);
    omega = dual.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);
    delta = dual.addVar(vtype=GRB.CONTINUOUS, lb = -GRB.INFINITY, ub=0);          
            
    
    for i,j in G.edges:
        if G.edges[i,j]['special'] == 1:
            dual.addConstr(alpha[i] - alpha[j] + theta[i,j] + pi[i,j] >= 0); 
            dual.addConstr(omega[i,j] - (1/M)*pi[i,j] >= 1);
        elif i!=t and j!=s:
            dual.addConstr(alpha[i] - alpha[j] + theta[i,j] >= 0);
        
    dual.addConstr(alpha[t] - alpha[s] + theta[t,s] + delta >= 0);         
    
    
    dual.setObjective(gp.quicksum(G.edges[i,j]['capacity']*(1-Gamma[i,j])*theta[i,j] for i,j in G.edges)+
                    gp.quicksum(omega[i,j]*G.edges[i,j]['special'] for i,j in G.edges)+
                    XBar[t,s]*delta, GRB.MINIMIZE);
    

    dual.setParam("IntegralityFocus",1);
    dual.setParam('TimeLimit', T_Limit); 
        
    dual.setParam("OutputFlag", 0);
    dual.update();
    dual.optimize();
    
    status = dual.status;
    
    if status == 2:
        print('\nDual Inner Problem Status: Optimal');
        print('Dual ObjVal = %g\n' %dual.objVal);
    else:
        print('\nDual Inner Problem Status: NOT Optimal');
        print('Dual Inner Problem Status = %g\n' %status);

In [8]:
def solver_Alternative (network, budget, T_Limit, summaryName):
    
    start_time = time.time(); 
    networkCSV = network+'.csv';

    # Reading network file
    with open(networkCSV, newline='') as f:
        reader = csv.reader(f);
        row1 = next(reader);
        s = int(row1[0]);             # Source node
        t = int(row1[1]);             # Sink node
        calA_level = int(row1[2]);    # Level of special arcs
    
        G = nx.DiGraph();
        data = pd.read_csv(networkCSV, skiprows=1, header=None);
        n_edge = len(data.index+1);
    
        for i in range(n_edge): 
            G.add_edge(data.iat[i,0], data.iat[i,1], capacity= data.iat[i,2], 
                    cost=data.iat[i,3], special=data.iat[i,4], level=data.iat[i,5]);  
    
    
    A = 0;
    U = G.edges[t,s]['capacity'];
    for i,j in G.edges:
        if G.edges[i,j]['special'] == 1:
            A = A + 1;
            
    ### ~~~~~~~~~~~~~~~~~~~~ SOLVING OPTIMISTIC PROBLEM (Optmistic LB and MIP Start)~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~  ####
    LB_opt = solver_optimistic (G, budget, s, t, A);
    
    ### ~~~~~~~~~~~~~~~~~~~~ # MFNIP MaxFlow (LB on xBar_ts)  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ###
    MFNIP_LB = MFNIP_Flow (network, budget, G, s, t); 
    
    ### ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ###
    
    ### ~~~~~~~~~~~~~~~~~~~~~~~~ MaxFlow without interdiction (UB on Xbar_ts) ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ####
    MF = MaxFlow (G, s, t);    
    
    ### ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~  ###
    
    M = A;
    
    instance = 'LogFile_'+network+'_b'+str(budget)+'.txt';
    
    if MF-MFNIP_LB >= 1:
        tau = range(math.ceil(math.log2(MF-MFNIP_LB))+1);
    else:
        tau = range(1);
    
    
    model = gp.Model("Alternative"); 
    
    gamma = model.addVars(G.edges, vtype=GRB.BINARY, name="gamma");
    
    x = model.addVars(G.edges, vtype=GRB.CONTINUOUS, name="x", lb = 0, ub = GRB.INFINITY);
    xBar = model.addVars(G.edges, vtype=GRB.CONTINUOUS, name="xBar", lb = 0, ub = GRB.INFINITY);
    z = model.addVars(G.edges, vtype=GRB.CONTINUOUS, name="z", lb = 0, ub = 1); 
    
    theta = model.addVars(G.edges, vtype=GRB.CONTINUOUS, name="theta", lb = 0, ub = GRB.INFINITY);
    alpha = model.addVars(G.nodes, vtype=GRB.CONTINUOUS, name="alpha", lb = -GRB.INFINITY, ub = GRB.INFINITY);  
    pi = model.addVars(G.edges, vtype=GRB.CONTINUOUS, name="pi", lb =-GRB.INFINITY, ub=0);
    omega = model.addVars(G.edges, vtype=GRB.CONTINUOUS, name="omega", lb = 0, ub = GRB.INFINITY);
    delta = model.addVar(vtype=GRB.CONTINUOUS, name="delat", lb = -GRB.INFINITY, ub=0);          
    
    lamb = model.addVars(tau, vtype=GRB.BINARY, name="lamb",);
    
    val = model.addVar(vtype=GRB.CONTINUOUS, lb = LB_opt, ub = A);      
    
    
    
    model.addConstr(gp.quicksum(gamma[i,j]*G.edges[i,j]['cost'] for i,j in G.edges) <= budget);   
    
    model.addConstrs(gp.quicksum(xBar[j,i] for i in G.successors(j))-
                     gp.quicksum(xBar[i,j] for i in G.predecessors(j)) == 0 for j in G.nodes);
            
    model.addConstrs(gp.quicksum(x[j,i] for i in G.successors(j))-
                        gp.quicksum(x[i,j] for i in G.predecessors(j)) == 0 for j in G.nodes);
                       
    
    
    for i,j in G.edges:
        model.addConstr(xBar[i,j] <= (1-gamma[i,j])*G.edges[i,j]['capacity']);
        model.addConstr(x[i,j] <= (1-gamma[i,j])*G.edges[i,j]['capacity']); 

        if G.edges[i,j]['special'] == 1:
            model.addConstr(alpha[i] - alpha[j] + theta[i,j] + pi[i,j] >= 0); 
            model.addConstr(x[i,j] >= (1/M)*z[i,j]);
            model.addConstr(omega[i,j] - (1/M)*pi[i,j] >= 1);
        elif i!=t and j!=s:
            model.addConstr(alpha[i] - alpha[j] + theta[i,j] >= 0);
        
    model.addConstr(alpha[t] - alpha[s] + theta[t,s] + delta >= 0);         
    
    model.addConstr(x[t,s] >= xBar[t,s]);
                                            
    model.addConstr(xBar[t,s] == MFNIP_LB + gp.quicksum(pow(2, tau[u])*lamb[u] for u in tau));
   
    model.addConstr(gp.quicksum(z[i,j]*G.edges[i,j]['special'] for i,j in G.edges) >= LB_opt);    
    
    model.addConstr(gp.quicksum(z[i,j]*G.edges[i,j]['special'] for i,j in G.edges)>=
                    gp.quicksum(G.edges[i,j]['capacity']*(1-gamma[i,j])*theta[i,j] for i,j in G.edges)
                    +gp.quicksum(omega[i,j]*G.edges[i,j]['special'] for i,j in G.edges)
                    +delta*(MFNIP_LB + gp.quicksum(pow(2, tau[u])*lamb[u] for u in tau)));

    
    model.setObjective(gp.quicksum(z[i,j]*G.edges[i,j]['special'] for i,j in G.edges), GRB.MINIMIZE);
    

    model.setParam("IntegralityFocus",1);
    model.setParam('TimeLimit', T_Limit); 
    
    #model.setParam('NonConvex', 2);
        
    model.setParam("LogToConsole", 0)
    model.setParam("OutputFlag", 1);
    model.setParam("LogFile", instance);
    model.update();
    model.optimize(); 
    
    
    status = model.status;
    
    end_time = time.time();
    run_time = round(end_time - start_time, 2);
    
    now = datetime.now();
    
    obj = model.objVal;
    sol = A - obj;
    OptGap = round(model.MIPGap, 2);
    
    Gamma = {};
    XBar = {};

    
    for i,j in G.edges:
        Gamma[i,j] = gamma[i,j].x;
        XBar[i,j] = xBar[i,j].x;
    

    
    ### Print to the screen and to result file #######################
    
    file = open('Alternative_NoWarmStart'+network+'_b'+str(budget)+'.txt', "w");
    file.write('Instance: %s, budget %g \n' %(network, budget));
    file.write('Instance executed on: %s \n\n' %now.strftime("%c"));
    
    file.write('Number of Nodes: %g \n' % (t+1));
    file.write('Number of Arcs: %g \n' % G.number_of_edges());
    file.write('Number of Special Arcs: %g \n' % A);
    file.write('Level of special set: %g \n' % calA_level);
    file.write('Budget: %g' % budget +'\n\n');
    
    
    print('\n');
    print('Instance: %s, budget %g' %(network, budget));   
    
    
    #### INNER PROBLEM AND DUAL ########################## 
    
    #Max_InnerProblem(G, s, t, M, Gamma, XBar);
    
    #Dual_InnerProblem(G, s, t, M, Gamma, XBar);
    
    ###########################################################
    
    
    if status == 2:
        print('Status: Optimal');
        print('Number of special arcs without flow: %g' % sol);
        print('Number of special arcs with flow: %g' % obj);
        
        file.write('Status: Optimal\n');
        file.write('Number of special arcs without flow: %g \n' % sol);
        file.write('Number of special arcs with flow: %g \n' % obj);
        
    elif status == 9:
        print('Status: Time Limit');
        print('Number of special arcs without flow: %g' % sol);
        print('Number of special arcs with flow: %g' % obj);     
        print('Optimality Gap: %g' % OptGap);
        
        file.write('Status: Time Limit \n');
        file.write('Number of special arcs without flow: %g \n' % sol);
        file.write('Number of special arcs with flow: %g \n' % obj);     
        file.write('Optimality Gap: %g \n' % OptGap);
              
    else:
        print('Status: %g' %status);
        file.write('Status: %g \n' % status);
    
    print('\nxBar[t,s]: %g' %xBar[t,s].x);
    file.write('\nxBar[t,s]: %g \n' %xBar[t,s].x);
    
    print('Flow: %g' %x[t,s].x);
    file.write('Flow: %g \n' %x[t,s].x);
    
    print('run time: %g sec \n' %run_time);
    file.write('run time: %g sec \n' %run_time);
    
    file.write('Level 0 Capacities: %g \n\n ' %G.edges[s,1]['capacity']);
    
    print('Intediction:')
    for i,j in G.edges:
        if gamma[i,j].x > 0.0001:
            print("arc (%g,%g), gamma = %g " %(i,j, gamma[i,j].x));
    print('\n');
    
    
    lev1 = 0;
    lev2 = 0;
    lev3 = 0;
    other_level = 0;
    
    file.write('\nInterdiction plan: \n');
    
    key1 = False;
    key2 = False;
    key3 = False;
    
    for i, j in G.edges: 
        if gamma[i,j].x > 0.0001:
            if G.edges[i,j]["level"] == 1:
                if key1 == False:
                    file.write("Level 1:" +'\n');
                    key1 = True;
                file.write("arc (%g,%g), gamma = %g \n" %(i,j, gamma[i,j].x));
                lev1 += 1;
            elif G.edges[i,j]["level"] == 2:
                if key2 == False:
                    file.write('Level 2: \n');
                    key2 = True;
                file.write("arc (%g,%g), gamma = %g \n" %(i,j, gamma[i,j].x));
                lev2 += 1;
            elif G.edges[i,j]["level"] == 3:
                if key3 == False:
                    file.write('Level 3: \n');
                    key3 = True;
                file.write("arc (%g,%g), gamma = %g \n" %(i,j, gamma[i,j].x));
                lev3 += 1;
            else:
                file.write('Other Levels: \n');
                file.write("arc (%g,%g), gamma = %g \n" %(i,j, gamma[i,j].x));
                other_level += 1;
                
    file.write('\nSpecial arcs with flow : \n');    
    print('Special arcs with flow :'); 
    for i,j in G.edges:
        if G.edges[i,j]['special'] == 1:
            if x[i,j].x > 0.00001:
                file.write('Arc (%s,%s), Flow: %f \n' %(i,j, x[i,j].x));
                print('Arc (%s,%s), Flow: %f ' %(i,j, x[i,j].x))

    
    print('\n');
    file.write('\n');
    file.close();
    
    rowFields = [network, budget, calA_level, (t+1), G.number_of_edges(), A, OptGap, round(obj), round(x[t,s].x,4),
                 lev1, lev2, lev3, other_level, run_time];
    
    with open(summaryName, 'a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile);
        csvwriter.writerow(rowFields);
        csvfile.close();

    

In [9]:
n_Networks = 2;
Budget = [3];

T_Limit = 7200;


summaryName = "Alternative_NoWarmStart_Summary.csv"
file_summary = open(summaryName, "w");
file_summary.write('Instance,Budget,Cal_A,Nodes,Arcs,SpecialArcs,OptGap,');
file_summary.write('ObjVal, Flow, Int_L1, Int_L2, Int_L3,Other_Lev,Time\n');
file_summary.close();

for n in range(1, n_Networks+1):
    network = 'Net'+str(n);
    
    for budget in Budget:
        solver_Alternative (network, budget, T_Limit, summaryName);

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-26
Set parameter IntegralityFocus to value 1
Optimistic LB: 11
run time: 0.03 sec 

Set parameter IntegralityFocus to value 1
Set parameter IntegralityFocus to value 1
Set parameter TimeLimit to value 7200


Instance: Net1, budget 3
Status: Optimal
Number of special arcs without flow: 7
Number of special arcs with flow: 12

xBar[t,s]: 208
Flow: 208.149
run time: 14.56 sec 

Intediction:
arc (1,8), gamma = 1 
arc (3,8), gamma = 1 
arc (5,10), gamma = 1 


Special arcs with flow :
Arc (6,13), Flow: 17.881884 
Arc (6,14), Flow: 22.910087 
Arc (6,15), Flow: 16.906476 
Arc (7,12), Flow: 21.904653 
Arc (7,13), Flow: 16.882838 
Arc (7,14), Flow: 18.911263 
Arc (7,15), Flow: 14.843945 
Arc (9,11), Flow: 14.062819 
Arc (9,12), Flow: 14.505700 
Arc (9,13), Flow: 21.488127 
Arc (9,14), Flow: 11.926852 
Arc (9,15), Flow: 15.924809 


Set parameter IntegralityFocus to value 1
Optimistic LB: 12
run time: 0.09 sec